In [1]:
import requests
from pprint import pprint
from lxml import html
from datetime import datetime
from pymongo import MongoClient

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client['news']
doc_news = db.doc_news

In [3]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'}

response = requests.get('https://yandex.ru/news/')

dom = html.fromstring(response.text)

In [4]:
items = dom.xpath("//article[contains(@class,'mg-card')]")

In [5]:
news = []
date = datetime.today().strftime('%d-%m-%Y')
for item in items:
    dict_news = {}
    
    link = item.xpath(".//@href")
    headline = item.xpath(".//h2[@class='mg-card__title']//text()")
    source = item.xpath(".//a[@class='mg-card__source-link']/text()")
    time = item.xpath(".//span[@class='mg-card-source__time']/text()")
    
    dict_news['source'] = source[0]
    dict_news['headline'] = headline[0].replace('\xa0', ' ')
    dict_news['link'] = link[0]
    dict_news['date'] = f'{date} {time[0]}' 


    news.append(dict_news)

In [6]:
for el in news:
    doc_news.insert_one(el)